In [19]:
import os
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [63]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc, _update_sp
from evaluation.fgc_eval import *

In [21]:
data = json_load("prediction/entity.json")
val_data = json_load(config.FGC_DEV)

In [22]:
get_SHINT(data)

100%|██████████| 247/247 [00:00<00:00, 15250.44it/s]

{'text': '\n阿佛烈·诺贝尔', 'start': 123, 'end': 131}
{'text': '阿佛烈·诺贝尔', 'start': 124, 'end': 131}
{'text': '阿佛烈·诺贝尔', 'start': 124, 'end': 131}
{'text': '10月', 'start': 495, 'end': 498}
{'text': '12月', 'start': 509, 'end': 512}
{'text': '他兴奋的把望远镜转向无垠的天空,观察月亮、银河以及灿烂的太阳，看到了许多以前的人所不知道的现象:月亮表面凹凸不平,像地球一样散布著深谷与断层;银河并非发光的云气,而是由无数星球所组成的;太阳的黑点，无论形状及大小都变化万千', 'start': 98, 'end': 209}
{'text': '藉著这些观察所得与精确的数学计算,伽利略深信之前波兰学者哥白尼提出的「地动说」—太阳静止不动,地球与行星围绕太阳运转,确是事实，而非一般人认为的数学假设', 'start': 212, 'end': 288}
{'text': '把他们在月球上活动的情形，传回地面', 'start': 159, 'end': 176}
{'text': '把他们在月球上活动的情形，传回地面', 'start': 159, 'end': 176}
{'text': '月球的颜色，随著太阳光线的角度变化，有时候是灰色，有时候是棕色或黄色。', 'start': 306, 'end': 341}


In [23]:
get_answer_sp(data, force=True)

100%|██████████| 247/247 [00:00<00:00, 27217.14it/s]


In [24]:
all_items, all_sp_predictions, all_answer_sp = eval_from_threshold(data, threshold=0.5)

100%|██████████| 247/247 [00:00<00:00, 99874.01it/s]


In [25]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)

{'sp_em': 0.205, 'sp_prec': 0.661, 'sp_recall': 0.651, 'sp_f1': 0.603}


In [26]:
metrics = eval_sp_fgc(all_answer_sp, all_sp_predictions)

{'sp_em': 0.18, 'sp_prec': 0.41, 'sp_recall': 0.633, 'sp_f1': 0.453}


In [27]:
len(all_answer_sp)

239

In [28]:
qid = []
qtext = []
shint_scores = []
sp = []
answer_sp = []
for d in data:
    for q in d['QUESTIONS']:
        qid.append(q['QID'])
        qtext.append(q['QTEXT_CN'])
        shint_score = [(shint_i, d['SENTS'][shint_i]['text'], q['sp_scores'][shint_i]) for shint_i in q['SHINT']]
        shint_scores.append(shint_score)
        sp_scores = [(sp_i, d['SENTS'][sp_i]['text'], q['sp_scores'][sp_i]) for sp_i in q['sp']]
        sp.append(sp_scores)

In [29]:
sp[0]

[(0, '苏轼（1037年1月8日－1101年8月24日），', 0.9120479226112366),
 (2, '北宋时著名的文学家、政治家、艺术家、医学家。', 0.6992460489273071),
 (18, '与唐代的古文运动发起者韩愈并称为「韩潮苏海」，', 0.6187727451324463)]

In [30]:
df = pd.DataFrame({'qid': qid, 'error_type':[0]*len(qid)})

In [31]:
df.head(5)

,qid,error_type
0,D001Q01,0
1,D001Q02,0
2,D001Q03,0
3,D001Q04,0
4,D001Q05,0


In [32]:
df.to_excel('entity_old.xlsx')

In [88]:
df = pd.read_excel('entity_old.xlsx')

In [90]:
df.error_type

0      4
1      4
2      1
3      4
4      3
      ..
242    0
243    0
244    0
245    0
246    0
Name: error_type, Length: 247, dtype: int64

In [91]:
error_count = [0]*7
for type_i in df.error_type:
    error_count[type_i] += 1

In [92]:
error_count

[208, 2, 4, 9, 13, 8, 3]

In [98]:
def print_sents_with_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                for s_i, s in enumerate(d['SENTS']):
#                     print((s_i, s['text'], q['sp_scores'][s_i]))
                    print("{}:{}".format(s_i, s['text'].strip()))

In [55]:
def count_string_in_d(text, qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            count = 0
            if q['QID'] == qid:
                for s_i, sent in enumerate(d['SENTS']):
                    if text in sent['text']:
                        print((s_i, sent['text']))
                        count += 1
                return count, len(d['SENTS'])

In [86]:
count_string_in_d('人类', 'D079Q01', val_data)

(22, '与人类同场较劲，')


(1, 51)

In [101]:
print_sents_with_qid('D013Q06', data)

0:高雄市，通称高雄（台湾话：Ko-hiông；客家话：Kô-hiùng），
1:简称「高」，
2:是中华民国的直辖市，
3:台湾六都之一。
4:其辖域东北至中央山脉及玉山主峰，
5:西南至南海上之南沙太平岛、中洲岛、东沙群岛。
6:顺时针方向与台南市及嘉义县、南投县、花莲县、台东县、屏东县、澎湖县（隔海邻接）等县市相邻。
7:面积达2952平方公里；
8:设籍人口277.3万人，
9:在台湾直辖市与县市中人口数排名第三。
10:高雄市自19世纪末开港，
11:旧称「打狗」，
12:发音来自于「竹林」的马卡道语 Takau，
13:音译近似于闽南语的「打狗」，
14:于是汉人便以「打狗」称之。
15:日治时代发展为重工业林立的港埠城市与军事要地，
16:素有「港都」之称，
17:并在20世纪中叶后成为台湾南部政治和经济及交通中枢，
18:台湾第二大的国际机场高雄国际机场、台湾最大港口高雄港及中央政府机关中华民国海洋委员会即位于本市。
19:2018年GaWC所公布的世界级城市名单中，
20:高雄市被列为「高度自足」等级之城市，
21:在台湾城市中排名第二，
22:仅次于台北。
23:在观光领域上，
24:高雄市被「孤独星球」（Lonely Planet）评为「2018全球最值得旅游的10大城市」中的第五位城市。


In [67]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("answer_sp:{}".format(q['answer_sp']))
                print("sp:{}".format(q['sp']))
                print("answer:{}".format(q['ANSWER']))
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print(q['sp_scores'][sp_i])
                    print_sent_ie(sp_i, d)
                    print()

In [81]:
print_analysis_from_qid('D073Q11', data)

D073Q11
菲律宾的鲸鲨保育政策为何?
atype:Misc
SHINT:[24]
answer_sp:[24]
sp:[22]
answer:[{'ATEXT': '禁止所有有關鯨鯊的商業獵捕、販賣、輸入與輸出活動。', 'ATEXT_CN': '禁止所有有关鲸鲨的商业猎捕、贩卖、输入与输出活动。', 'ATOKEN': [{'text': '禁止所有有關鯨鯊的商業獵捕', 'text_cn': '禁止所有有关鲸鲨的商业猎捕', 'start': 539, 'end': 552}, {'text': '販賣', 'text_cn': '贩卖', 'start': 553, 'end': 555}, {'text': '輸入', 'text_cn': '输入', 'start': 556, 'end': 558}, {'text': '輸出活動。', 'text_cn': '输出活动。', 'start': 559, 'end': 564}]}]
0.4599601626396179
sentence22

鲸鲨的保育工作：马尔地夫在1995年开始禁止猎捕鲸鲨，
{'id': 'D0-S0-M0', 'string': '马尔地夫', 'type': 'PER', 'char_b': 9, 'char_e': 13}
{'id': 'D0-S0-M1', 'string': '1995年', 'type': 'DATE', 'char_b': 14, 'char_e': 19}

0.20972082018852234
sentence23
是最早针对鲸鲨进行全面保护的国家。

0.3648209571838379
sentence24

菲律宾在1998年禁止所有有关鲸鲨的商业猎捕、贩卖、输入与输出活动。
{'id': 'D0-S0-M0', 'string': '菲律宾', 'type': 'COUNTRY', 'char_b': 1, 'char_e': 4}
{'id': 'D0-S0-M1', 'string': '1998年', 'type': 'DATE', 'char_b': 5, 'char_e': 10}



In [20]:
find_q('D061Q01', data)

{'DID': 'D001', 'QUESTIONS': [{'QID': 'D001Q07', 'QTYPE': '進階題', 'ATYPE': 'YesNo', 'AMODE': ['YesNo'], 'QTEXT': '蘇東坡與韓愈是否為好朋友?', 'QTEXT_CN': '苏东坡与韩愈是否为好朋友?', 'SENTS': [{'text': '苏东坡与韩愈是否为好朋友?', 'start': 0, 'end': 13, 'IE': {'NER': [{'id': 'D0-S0-M0', 'string': '苏东坡', 'type': 'PER', 'char_b': 0, 'char_e': 3}, {'id': 'D0-S0-M1', 'string': '韩愈', 'type': 'PER', 'char_b': 4, 'char_e': 6}], 'COREF': {}, 'RELATION': [], 'TOKEN': [{'word': '苏东坡', 'char_b': 0, 'char_e': 3, 'pos': 'NR'}, {'word': '与', 'char_b': 3, 'char_e': 4, 'pos': 'CC'}, {'word': '韩愈', 'char_b': 4, 'char_e': 6, 'pos': 'NR'}, {'word': '是否', 'char_b': 6, 'char_e': 8, 'pos': 'VV'}, {'word': '为好', 'char_b': 8, 'char_e': 10, 'pos': 'VV'}, {'word': '朋友', 'char_b': 10, 'char_e': 12, 'pos': 'NN'}, {'word': '?', 'char_b': 12, 'char_e': 13, 'pos': 'PU'}]}}], 'SHINT': [0, 2, 4, 18], 'ANSWER': [{'ATEXT': '否', 'ATEXT_CN': '否', 'ATOKEN': [{'text': '否', 'text_cn': '否', 'start': -1, 'end': 0}]}], 'ASPAN': [{'text': '唐代', 'start': 250, 'end':

In [21]:
data[100]

{'DID': 'D091',
 'QUESTIONS': [{'QID': 'D091Q11',
   'QTYPE': '基礎題',
   'ATYPE': 'Kinship',
   'AMODE': ['Kinship'],
   'QTEXT': '先生的奶奶要怎麼稱呼她?',
   'QTEXT_CN': '先生的奶奶要怎么称呼她?',
   'SENTS': [{'text': '先生的奶奶要怎么称呼她?',
     'start': 0,
     'end': 12,
     'IE': {'NER': [],
      'COREF': {},
      'RELATION': [],
      'TOKEN': [{'word': '先生', 'char_b': 0, 'char_e': 2, 'pos': 'NN'},
       {'word': '的', 'char_b': 2, 'char_e': 3, 'pos': 'DEG'},
       {'word': '奶奶', 'char_b': 3, 'char_e': 5, 'pos': 'NN'},
       {'word': '要', 'char_b': 5, 'char_e': 6, 'pos': 'VV'},
       {'word': '怎么', 'char_b': 6, 'char_e': 8, 'pos': 'AD'},
       {'word': '称呼', 'char_b': 8, 'char_e': 10, 'pos': 'VV'},
       {'word': '她', 'char_b': 10, 'char_e': 11, 'pos': 'PN'},
       {'word': '?', 'char_b': 11, 'char_e': 12, 'pos': 'PU'}]}}],
   'SHINT': [2, 3],
   'ANSWER': [{'ATEXT': '母親、媽媽、娘及婆婆',
     'ATEXT_CN': '母亲、妈妈、娘及婆婆',
     'ATOKEN': [{'text': '母親', 'text_cn': '母亲', 'start': 144, 'end': 146},
      {'text':

In [22]:
d = data[100]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("answer_sp:{}".format(q['answer_sp']))
    print("sp:{}".format(q['sp']))
    print()
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print(q['sp_scores'][sp_i])
        print(get_sent_ie(sp_i, d))

D091Q11
先生的奶奶要怎么称呼她?
atype:Kinship
SHINT:[2, 3]
answer_sp:[11, 12]
sp:[3, 38]

0.009744958020746708
sentence2
当对象是丈夫的祖母，

0.8515264987945557
sentence3
我们要称呼她为祖姑(奶奶)。

0.013354835100471973
sentence4
而我们自己要称作是孙媳，

0.00023756733571644872
sentence5
对第三者介绍他们时，
{'id': 'D0-S0-M0', 'string': '第三', 'type': 'ORDINAL', 'char_b': 1, 'char_e': 3}

0.0014695506542921066
sentence6
会说家祖翁家祖姑。
{'id': 'D0-S0-M0', 'string': '家祖翁', 'type': 'PER', 'char_b': 2, 'char_e': 5}

0.00027178795426152647
sentence7
当我们称呼别人的家祖翁家祖姑时，

0.0010948771378025413
sentence8
会说令祖翁令祖姑。

0.0015545290661975741
sentence9

当对象是丈夫的父亲，

0.00352524034678936
sentence10
我们要称呼他为父亲(爸爸/爹/公公/老爷)；

0.1203518956899643
sentence11
当对象是丈夫的母亲，

0.47855305671691895
sentence12
我们要称呼她为母亲(妈妈/娘/婆婆/奶奶(广东人用))。
{'id': 'D0-S0-M0', 'string': '广东人', 'type': 'DEMONYM', 'char_b': 21, 'char_e': 24}

0.005490172654390335
sentence13
而我们自己要称作是媳，

0.00023756733571644872
sentence14
对第三者介绍他们时，
{'id': 'D0-S0-M0', 'string': '第三', 'type': 'ORDINAL', 'char_b': 1, 'char_

In [23]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("sent_len:{}".format(len(d['SENTS'])))
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("answer_sp:{}".format(q['answer_sp']))
                print("sp:{}".format(q['sp']))
                print()
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print(q['sp_scores'][sp_i])
                    print(get_sent_ie(sp_i, d))

In [24]:
print_analysis_from_qid('D031Q01', data)

D031Q01
新版的北美贸易协定英文简称为何?
sent_len:15
atype:Object
SHINT:[7, 14]
answer_sp:[14]
sp:[3]

0.36715084314346313
sentence3
并阻碍新的北美自由贸易协定达成。
{'id': 'D0-S0-M0', 'string': '北美', 'type': 'LOC', 'char_b': 5, 'char_e': 7}

0.0005357848713174462
sentence4

法新社报导，
{'id': 'D0-S0-M0', 'string': '法新社', 'type': 'ORG', 'char_b': 1, 'char_e': 4}

0.005978924222290516
sentence5
川普在华盛顿哥伦比亚特区的一项活动指出：「我很高兴地宣布，我们已与加拿大和墨西哥达成协议，将出售我们的产品至这些国家而无需课征关税。」
美国去年以国家安全为由，
{'id': 'D0-S0-M0', 'string': '华盛顿哥伦比亚特区', 'type': 'CITY', 'char_b': 3, 'char_e': 12}
{'id': 'D0-S0-M1', 'string': '一', 'type': 'NUMBER', 'char_b': 13, 'char_e': 14}
{'id': 'D0-S0-M2', 'string': '高兴', 'type': 'PER', 'char_b': 23, 'char_e': 25}
{'id': 'D0-S0-M3', 'string': '加拿大', 'type': 'COUNTRY', 'char_b': 33, 'char_e': 36}
{'id': 'D0-S0-M4', 'string': '墨西哥', 'type': 'COUNTRY', 'char_b': 37, 'char_e': 40}
{'id': 'D0-S1-M0', 'string': '美国', 'type': 'COUNTRY', 'char_b': 68, 'char_e': 70}
{'id': 'D0-S1-M1', 'string': '去年', 'type': 'DATE', 'char_b': 70, 'ch

In [25]:
len(data)

247

In [ ]:
f = open('prediction/entitiy.analysis', 'w')
f.write()
f.close()